**BERT for question answering NOT fine-tuned**

source:   https://huggingface.co/docs/transformers/master/task_summary

In [1]:
# code breaker
# the purpose of this cell is to break the code
# so the whole Collab notebook does not run accidentally
# print(break_the_code)

**Key steps to prepare the BERT model to be used**

In [2]:
# Step 1: install transformer
!pip install -q transformers

In [3]:
# Step 2: import BERT model and BERT tokenizer
from transformers import BertForQuestionAnswering, BertTokenizer

In [4]:
# Step 3: get the model and the tokenizer
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
tokenizer_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(tokenizer_name)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# Step 4: torch must be imported
import torch

In [6]:
# Step 5: function to answer question based on provided context
def answerQuestion(question, context):
  inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt")
  input_ids = inputs["input_ids"].tolist()[0]
  outputs = model(**inputs)
  answer_start_scores = outputs.start_logits
  answer_end_scores = outputs.end_logits
  # get the most likely beginning of answer with the argmax of the score
  answer_start = torch.argmax(answer_start_scores)
  # get the most likely end of answer with the argmax of the score
  answer_end = torch.argmax(answer_end_scores) + 1
  answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )
  print(f"Answer: {answer}")

**Steps to prepare datasets - optional**

In [7]:
# mount drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
# set path to datasets
path = "/content/gdrive/My Drive/Models_BERT_GPT2/"
print(path)

/content/gdrive/My Drive/Models_BERT_GPT2/


In [9]:
# import pandas and numpy
import pandas as pd
import numpy as np

In [10]:
# import regex to remove the \n \t \r
import re
regex = re.compile(r'[\n\r\t]')

In [11]:
# Alternative Products dataset

# variable
tempText = ""

# read file
file = open(path+"AlternativeProductsCleaned.txt", "r")
line = file.read()

# prepare each line
tempText += line

# gesplit temporary texdt tempText into tokens
tokens = tempText.split(':')

# product and answer
product = ""
answer = ""

# index used when looping through tokens
index = 0
# index used when inserting data into dataset array
datasetIndex = 0

# alternativeProductsData is used to collect data
alternativeProductsData = []

# loop thorugh tokens
for token in tokens:

  # line is contains keyword, product to be replaced
  if index%2==0:
    token = token.replace("'","")
    token = regex.sub(' ', token)
    product = token.strip()
    question = "What can I use instead of "+product+"?"
    # print(question)

  # line contains alternative products
  elif index%2!=0:
    #answer = token.replace(", do you have any of these ingredients?","")
    alternatives = regex.sub(' ', token)
    alternatives = alternatives.strip()
    context = "If you do not have "+product+" you can use instead "+alternatives
    answer = alternatives
    # print(context)
    # print(answer)

  # add question and text to the dataset
  if index>0 and index%2!=0:
    alternativeProductsData.append({"context":context,"question":question})
    datasetIndex = datasetIndex + 1

  # increase index
  index = index+ 1

# global alternativeProducts
alternativeProducts = pd.DataFrame(alternativeProductsData)

for i in range (len(alternativeProducts)):
    context = alternativeProducts["context"][i]
    question = alternativeProducts["question"][i]
    print(i+1)
    print(context)
    print(question)
    print("")

1
If you do not have plain flour you can use instead oat flour, bread flour, cake flour or coconut flour
What can I use instead of plain flour?

2
If you do not have olive oil you can use instead peanut oil, butter, coconut milk, ghee, walnut oil, sunflower oil, canola oil or vegetable oil
What can I use instead of olive oil?

3
If you do not have salt you can use instead mint, rosemary, nutmeg, basil, cardamon, chili, cinnamon or chives
What can I use instead of salt?

4
If you do not have black pepper you can use instead white pepper, cayenne pepper or papaya seeds
What can I use instead of black pepper?

5
If you do not have eggs you can use instead commerical egg substitute or tofu
What can I use instead of eggs?

6
If you do not have milk you can use instead cream, powerdered milk, plain yogurt, nut milk, soy milk, oat milk or rice milk
What can I use instead of milk?

7
If you do not have sausage you can use instead vegetarian sausages
What can I use instead of sausage?

8
If you

In [12]:
# saving the alternativeProducts as csv to the google drive
# alternativeProductsDestiation = '/content/gdrive/My Drive/HonoursProject/preparedAlternativeProducts.csv'
# with open(alternativeProductsDestiation, 'w', encoding = 'utf-8-sig') as f:
  # alternativeProducts.to_csv(f)

In [13]:
# check BERT capabilities to answer questions about the alternative products

# for i in range (len(alternativeProducts)):
#    context = alternativeProducts["context"][i]
#    context = ""
#    question = alternativeProducts["question"][i]
#    print(i+1)
#    print("Context:"+" "+context)
#    answerQuestion(question, context)
#    print("")

In [14]:
# Utensils Description dataset

# variable
tempText = ""

# read file
file = open(path+"UtensilsDescriptionCleaned.txt", "r")
line = file.read()

# prepare each line
tempText += line

# gesplit temporary texdt tempText into tokens
tokens = tempText.split(':')

# keyword and answer
keyword = ""
answer = ""

# index used when looping through tokens
index = 0
# index used when inserting data into dataset array
datasetIndex = 0

# utensilsDescriptionData is used to collect data
utensilsDescriptionData = []

# loop thorugh tokens
for token in tokens:

  # line is contains keyword, product to be replaced
  if index%2==0:
    token = token.replace("'","")
    token = regex.sub(' ', token)
    keyword = token.strip()
    question = "What is  "+keyword+"?"
    # print(question)

  # line contains alternative products
  elif index%2!=0:
    #answer = token.replace(", do you have any of these ingredients?","")
    answer = regex.sub(' ', token)
    answer = answer.strip()
    context = keyword+" is "+answer +"."
    # print(context)

  # add question and text to the dataset
  if index>0 and index%2!=0:
    utensilsDescriptionData.append({"context":context,"question":question})
    datasetIndex = datasetIndex + 1

  # increase index
  index = index+ 1

# global alternativeProducts
utensilsDescription = pd.DataFrame(utensilsDescriptionData)

for i in range (len(utensilsDescription)):
    context = utensilsDescription["context"][i]
    question = utensilsDescription["question"][i]
    print(i+1)
    print(question)
    print(context)
    print("")

1
What is  bowl?
bowl is a cooking container that is usually larger than a cup and kept in a cupboard in the kitchen, it is typically made from glass, ceramic, plastic..

2
What is  spoon?
spoon is an eating or cooking utensil consisting of a small shallow bowl with a relatively long handle made from metal, plastic, wood it is usually kept in the kitchen, in drawer..

3
What is  sieve?
sieve is a metal or plastic device with meshes or perforations through which finer particles of a mixture are sifted, it is usually kept in a kitchen and stored in a cupboard..

4
What is  pan?
pan is a shallow and open cooking container made from metal, it is usually found in a kitchen and stored in a cupboard..

5
What is  baking tray?
baking tray is a metal rectangular sheet with a rolled edge used for baking, it is usually found in a kitchen inside a cupboard or draw..

6
What is  whisk?
whisk is a metal or plastic wire utensil used for beating food by hand and is usually found in a kitchen inside a 

In [15]:
# saving the utensilsDescription as csv to the google drive
# utensilsDescriptionDestination = '/content/gdrive/My Drive/HonoursProject/preparedUtensilsDescription.csv'
# with open(utensilsDescriptionDestination, 'w', encoding = 'utf-8-sig') as f:
  # utensilsDescription.to_csv(f)

In [16]:
# check BERT capabilities to answer questions about the utensils description
# for i in range (len(utensilsDescription)):
#    question = utensilsDescription["question"][i]
#    text = utensilsDescription["text"][i]
#    print(i+1)
#    answerQuestion(question, text)
#    print("Text:"+" "+text)
#    print("")

In [17]:
# alternative products

**Checking BERT capabilities of showing common-sense**

**Alternative Products**

In [18]:
# question 1 - plain flour
# source: https://en.wikipedia.org/wiki/Flour#Plain_or_all-purpose_flour

question = "What can be used instead of plain flour?"
context = "All-purpose, or AP flour, or plain flour is medium in gluten protein content at 9.5-11.5%[11] (10-12% from second source[12]) protein content. It has adequate protein content for many bread and pizza bases, though bread flour and special 00 grade Italian flour are often preferred for these purposes, respectively, especially by artisan bakers. Some biscuits are also prepared using this type of flour. Plain refers not only to AP flour's middling gluten content but also to its lack of any added leavening agent (as in self-rising flour)."
answer = "oat flour, bread flour, cake flour or coconut flour"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")



Question: What can be used instead of plain flour?
Context: All-purpose, or AP flour, or plain flour is medium in gluten protein content at 9.5-11.5%[11] (10-12% from second source[12]) protein content. It has adequate protein content for many bread and pizza bases, though bread flour and special 00 grade Italian flour are often preferred for these purposes, respectively, especially by artisan bakers. Some biscuits are also prepared using this type of flour. Plain refers not only to AP flour's middling gluten content but also to its lack of any added leavening agent (as in self-rising flour).There are other products that can be used instead. For instance: oat flour, bread flour, cake flour or coconut flour.
Answer: oat flour, bread flour, cake flour or coconut flour
Answer generated by BERT
Answer: oat flour , bread flour , cake flour or coconut flour



In [19]:
# question 2 - olive oil
# source: https://en.wikipedia.org/wiki/Olive_oil

question = "What can be used instead of olive oil?"
context = "Olive oil is a liquid fat obtained from olives (the fruit of Olea europaea; family Oleaceae), a traditional tree crop of the Mediterranean Basin, produced by pressing whole olives and extracting the oil. It is commonly used in cooking, for frying foods or as a salad dressing. It is also used in cosmetics, pharmaceuticals, and soaps, and as a fuel for traditional oil lamps, and has additional uses in some religions. The olive is one of three core food plants in Mediterranean cuisine; the other two are wheat and grapes. Olive trees have been grown around the Mediterranean since the 8th millennium BC."
answer = "peanut oil, butter, coconut milk, ghee, walnut oil, sunflower oil, canola oil or vegetable oil"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")



Question: What can be used instead of olive oil?
Context: Olive oil is a liquid fat obtained from olives (the fruit of Olea europaea; family Oleaceae), a traditional tree crop of the Mediterranean Basin, produced by pressing whole olives and extracting the oil. It is commonly used in cooking, for frying foods or as a salad dressing. It is also used in cosmetics, pharmaceuticals, and soaps, and as a fuel for traditional oil lamps, and has additional uses in some religions. The olive is one of three core food plants in Mediterranean cuisine; the other two are wheat and grapes. Olive trees have been grown around the Mediterranean since the 8th millennium BC.There are other products that can be used instead. For instance: peanut oil, butter, coconut milk, ghee, walnut oil, sunflower oil, canola oil or vegetable oil.
Answer: peanut oil, butter, coconut milk, ghee, walnut oil, sunflower oil, canola oil or vegetable oil
Answer generated by BERT
Answer: peanut oil , butter , coconut milk , ghe

In [20]:
# question 3 - salt
# source: https://en.wikipedia.org/wiki/Salt

question = "What can be used instead of salt?"
context = "Salt is a mineral composed primarily of sodium chloride (NaCl), a chemical compound belonging to the larger class of salts; salt in the form of a natural crystalline mineral is known as rock salt or halite. Salt is present in vast quantities in seawater. The open ocean has about 35 g (1.2 oz) of solids per liter of sea water, a salinity of 3.5%. Salt is essential for life in general, and saltiness is one of the basic human tastes. Salt is one of the oldest and most ubiquitous food seasonings, and salting is an important method of food preservation."
answer = "mint, rosemary, nutmeg, basil, cardamon, chili, cinnamon or chives"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")



Question: What can be used instead of salt?
Context: Salt is a mineral composed primarily of sodium chloride (NaCl), a chemical compound belonging to the larger class of salts; salt in the form of a natural crystalline mineral is known as rock salt or halite. Salt is present in vast quantities in seawater. The open ocean has about 35 g (1.2 oz) of solids per liter of sea water, a salinity of 3.5%. Salt is essential for life in general, and saltiness is one of the basic human tastes. Salt is one of the oldest and most ubiquitous food seasonings, and salting is an important method of food preservation.There are other products that can be used instead. For instance: mint, rosemary, nutmeg, basil, cardamon, chili, cinnamon or chives.
Answer: mint, rosemary, nutmeg, basil, cardamon, chili, cinnamon or chives
Answer generated by BERT
Answer: mint , rosemary , nutmeg , basil , cardamon , chili , cinnamon or chives



In [21]:
# question 4 - black pepper
# source: https://en.wikipedia.org/wiki/Black_pepper

question = "What can be used instead of black pepper?"
context = "Black pepper (Piper nigrum) is a flowering vine in the family Piperaceae, cultivated for its fruit, known as a peppercorn, which is usually dried and used as a spice and seasoning. The fruit is a drupe (stonefruit) which is about 5 mm (0.20 in) in diameter (fresh and fully mature), dark red, and contains a stone which encloses a single pepper seed. Peppercorns and the ground pepper derived from them may be described simply as pepper, or more precisely as black pepper (cooked and dried unripe fruit), green pepper (dried unripe fruit), or white pepper (ripe fruit seeds)."
answer = "white pepper, cayenne pepper or papaya seeds"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of black pepper?
Context: Black pepper (Piper nigrum) is a flowering vine in the family Piperaceae, cultivated for its fruit, known as a peppercorn, which is usually dried and used as a spice and seasoning. The fruit is a drupe (stonefruit) which is about 5 mm (0.20 in) in diameter (fresh and fully mature), dark red, and contains a stone which encloses a single pepper seed. Peppercorns and the ground pepper derived from them may be described simply as pepper, or more precisely as black pepper (cooked and dried unripe fruit), green pepper (dried unripe fruit), or white pepper (ripe fruit seeds).There are other products that can be used instead. For instance: white pepper, cayenne pepper or papaya seeds.
Answer: white pepper, cayenne pepper or papaya seeds
Answer generated by BERT
Answer: papaya seeds



In [22]:
# question 5 - eggs
# source: https://en.wikipedia.org/wiki/Egg_as_food

question = "What can be used instead of eggs?"
context = "Eggs are laid by female animals of many different species, including birds, reptiles, amphibians, a few mammals, and fish, and many of these have been eaten by humans for thousands of years.[1] Bird and reptile eggs consist of a protective eggshell, albumen (egg white), and vitellus (egg yolk), contained within various thin membranes. The most commonly consumed eggs are chicken eggs. Other poultry eggs including those of duck and quail also are eaten. Fish eggs are called roe and caviar."
answer = "commercial egg substitute or tofu"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of eggs?
Context: Eggs are laid by female animals of many different species, including birds, reptiles, amphibians, a few mammals, and fish, and many of these have been eaten by humans for thousands of years.[1] Bird and reptile eggs consist of a protective eggshell, albumen (egg white), and vitellus (egg yolk), contained within various thin membranes. The most commonly consumed eggs are chicken eggs. Other poultry eggs including those of duck and quail also are eaten. Fish eggs are called roe and caviar.There are other products that can be used instead. For instance: commercial egg substitute or tofu.
Answer: commercial egg substitute or tofu
Answer generated by BERT
Answer: commercial egg substitute or tofu



In [23]:
# question 6 - milk
# source: https://en.wikipedia.org/wiki/Milk

question = "What can be used instead of milk?"
context = "Milk is a nutrient-rich liquid food produced by the mammary glands of mammals. It is the primary source of nutrition for young mammals (including breastfed human infants) before they are able to digest solid food.[1] Early-lactation milk, which is called colostrum, contains antibodies that strengthen the immune system and thus reduces the risk of many diseases. Milk contains many other nutrients,[2] including protein and lactose. While Humans are the primary consumers of other mammal's milk,[3][4]other interspecies consumption of milk has been observed.[5][6]"
answer = "cream, powerdered milk, plain yogurt, nut milk, soy milk, oat milk or rice milk"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of milk?
Context: Milk is a nutrient-rich liquid food produced by the mammary glands of mammals. It is the primary source of nutrition for young mammals (including breastfed human infants) before they are able to digest solid food.[1] Early-lactation milk, which is called colostrum, contains antibodies that strengthen the immune system and thus reduces the risk of many diseases. Milk contains many other nutrients,[2] including protein and lactose. While Humans are the primary consumers of other mammal's milk,[3][4]other interspecies consumption of milk has been observed.[5][6]There are other products that can be used instead. For instance: cream, powerdered milk, plain yogurt, nut milk, soy milk, oat milk or rice milk.
Answer: cream, powerdered milk, plain yogurt, nut milk, soy milk, oat milk or rice milk
Answer generated by BERT
Answer: other products that can be used instead . for instance : cream , powerdered milk , plain yogurt , nut milk , soy mi

In [24]:
# question 7 - sausage
# source: https://en.wikipedia.org/wiki/Sausage

question = "What can be used instead of sausage?"
context = "A sausage is a type of meat product usually made from ground meat, often pork, beef, or poultry, along with salt, spices and other flavourings. Other ingredients such as grains or breadcrumbs may be included as fillers or extenders. When used as an adjective, the word sausage can refer to the loose sausage meat, which can be formed into patties or stuffed into a skin. When referred to as a sausage, the product is usually cylindrical and encased in a skin."
answer = "vegetarian sausages"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of sausage?
Context: A sausage is a type of meat product usually made from ground meat, often pork, beef, or poultry, along with salt, spices and other flavourings. Other ingredients such as grains or breadcrumbs may be included as fillers or extenders. When used as an adjective, the word sausage can refer to the loose sausage meat, which can be formed into patties or stuffed into a skin. When referred to as a sausage, the product is usually cylindrical and encased in a skin.There are other products that can be used instead. For instance: vegetarian sausages.
Answer: vegetarian sausages
Answer generated by BERT
Answer: vegetarian sausages



In [25]:
# question 8 - mustard
# source: https://en.wikipedia.org/wiki/Mustard_(condiment)

question = "What can be used instead of mustard?"
context = "Mustard is a condiment made from the seeds of a mustard plant (white/yellow mustard, Sinapis alba; brown mustard, Brassica juncea; or black mustard, Brassica nigra). The whole, ground, cracked, or bruised mustard seeds are mixed with water, vinegar, lemon juice, wine, or other liquids, salt, and often other flavourings and spices, to create a paste or sauce ranging in colour from bright yellow to dark brown. The seed itself has a strong, pungent, and somewhat bitter taste. The taste of mustard condiments ranges from sweet to spicy.[1][failed verification][unreliable source]"
answer = "turmeric, wasabi powder or horseradish powder"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of mustard?
Context: Mustard is a condiment made from the seeds of a mustard plant (white/yellow mustard, Sinapis alba; brown mustard, Brassica juncea; or black mustard, Brassica nigra). The whole, ground, cracked, or bruised mustard seeds are mixed with water, vinegar, lemon juice, wine, or other liquids, salt, and often other flavourings and spices, to create a paste or sauce ranging in colour from bright yellow to dark brown. The seed itself has a strong, pungent, and somewhat bitter taste. The taste of mustard condiments ranges from sweet to spicy.[1][failed verification][unreliable source]There are other products that can be used instead. For instance: turmeric, wasabi powder or horseradish powder.
Answer: turmeric, wasabi powder or horseradish powder
Answer generated by BERT
Answer: turmeric , wasabi powder or horseradish powder



In [26]:
# question 9 - beef stock
# source: https://en.wikipedia.org/wiki/Stock_(food)

question = "What can be used instead of beef stock?"
context = "Stock, sometimes called bone broth, is a savory cooking liquid that forms the basis of many dishes, particularly soups, stews and sauces. Making stock involves simmering animal bones, meat, seafood, or vegetables in water or wine, often for an extended period. Mirepoix or other aromatics may be added for more flavor. Bones: Beef and chicken bones are most commonly used; fish is also common. The flavor of the stock comes from the bone marrow, cartilage and other connective tissue. Connective tissue contains collagen, which is converted into gelatin that thickens the liquid. Stock made from bones needs to be simmered for long periods; pressure cooking methods shorten the time necessary to extract the flavor from the bones."
answer = "vegetable stock, chicken stock, bouillion stock or red wine"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of beef stock?
Context: Stock, sometimes called bone broth, is a savory cooking liquid that forms the basis of many dishes, particularly soups, stews and sauces. Making stock involves simmering animal bones, meat, seafood, or vegetables in water or wine, often for an extended period. Mirepoix or other aromatics may be added for more flavor. Bones: Beef and chicken bones are most commonly used; fish is also common. The flavor of the stock comes from the bone marrow, cartilage and other connective tissue. Connective tissue contains collagen, which is converted into gelatin that thickens the liquid. Stock made from bones needs to be simmered for long periods; pressure cooking methods shorten the time necessary to extract the flavor from the bones.There are other products that can be used instead. For instance: vegetable stock, chicken stock, bouillion stock or red wine.
Answer: vegetable stock, chicken stock, bouillion stock or red wine
Answer generated 

In [27]:
# question 10 - cabbage
# source: https://en.wikipedia.org/wiki/Cabbage

question = "What can be used instead of cabbage?"
context = "Cabbage, comprising several cultivars of Brassica oleracea, is a leafy green, red (purple), or white (pale green) biennial plant grown as an annual vegetable crop for its dense-leaved heads. It is descended from the wild cabbage (B. oleracea var. oleracea), and belongs to the cole crops or brassicas, meaning it is closely related to broccoli and cauliflower (var. botrytis); Brussels sprouts (var. gemmifera); and Savoy cabbage (var. sabauda)."
answer = "brussels sprouts, yu choy, kangkong or bok choy"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of cabbage?
Context: Cabbage, comprising several cultivars of Brassica oleracea, is a leafy green, red (purple), or white (pale green) biennial plant grown as an annual vegetable crop for its dense-leaved heads. It is descended from the wild cabbage (B. oleracea var. oleracea), and belongs to the cole crops or brassicas, meaning it is closely related to broccoli and cauliflower (var. botrytis); Brussels sprouts (var. gemmifera); and Savoy cabbage (var. sabauda).There are other products that can be used instead. For instance: brussels sprouts, yu choy, kangkong or bok choy
Answer: brussels sprouts, yu choy, kangkong or bok choy
Answer generated by BERT
Answer: 



In [28]:
# question 11 - garlic
# source: https://en.wikipedia.org/wiki/Garlic

question = "What can be used instead of garlic?"
context = "Garlic (Allium sativum) is a species of bulbous flowering plant in the genus Allium. Its close relatives include the onion, shallot, leek, chive,[2] Welsh onion and Chinese onion.[3] It is native to Central Asia and northeastern Iran and has long been a common seasoning worldwide, with a history of several thousand years of human consumption and use.[4][5] It was known to ancient Egyptians and has been used as both a food flavoring and a traditional medicine.[6][7] China produces 76% of the world's supply of garlic.[8]"
answer = "garlic oil, onion, chives or lemon zest"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of garlic?
Context: Garlic (Allium sativum) is a species of bulbous flowering plant in the genus Allium. Its close relatives include the onion, shallot, leek, chive,[2] Welsh onion and Chinese onion.[3] It is native to Central Asia and northeastern Iran and has long been a common seasoning worldwide, with a history of several thousand years of human consumption and use.[4][5] It was known to ancient Egyptians and has been used as both a food flavoring and a traditional medicine.[6][7] China produces 76% of the world's supply of garlic.[8]There are other products that can be used instead. For instance: garlic oil, onion, chives or lemon zest.
Answer: garlic oil, onion, chives or lemon zest
Answer generated by BERT
Answer: garlic oil , onion , chives or lemon zest



In [29]:
# question 12 - mixed herbs
# source: https://en.wikipedia.org/wiki/Herb

question = "What can be used instead of mixed herbs?"
context = "In general use, herbs are a widely distributed and widespread group of plants, excluding vegetables and other plants consumed for macronutrients, with savory or aromatic properties that are used for flavoring and garnishing food, for medicinal purposes, or for fragrances. Culinary use typically distinguishes herbs from spices. Herbs generally refers to the leafy green or flowering parts of a plant (either fresh or dried), while spices are usually dried and produced from other parts of the plant, including seeds, bark, roots and fruits."
answer = "basil, marjoram, min, oregano or parsley"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of mixed herbs?
Context: In general use, herbs are a widely distributed and widespread group of plants, excluding vegetables and other plants consumed for macronutrients, with savory or aromatic properties that are used for flavoring and garnishing food, for medicinal purposes, or for fragrances. Culinary use typically distinguishes herbs from spices. Herbs generally refers to the leafy green or flowering parts of a plant (either fresh or dried), while spices are usually dried and produced from other parts of the plant, including seeds, bark, roots and fruits.There are other products that can be used instead. For instance: basil, marjoram, min, oregano or parsley.
Answer: basil, marjoram, min, oregano or parsley
Answer generated by BERT
Answer: other products



In [30]:
# question 13 - butter
# source: https://en.wikipedia.org/wiki/Butter

question = "What can be used instead of butter?"
context = "Butter is a dairy product made from the fat and protein components of churned cream. It is a semi-solid emulsion at room temperature, consisting of approximately 80% butterfat. It is used at room temperature as a spread, melted as a condiment, and used as an ingredient in baking, sauce making, pan frying, and other cooking procedures."
answer = "olive oil, ghee, greek yogurt, avocado, pumpkin puree or coconut oil"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of butter?
Context: Butter is a dairy product made from the fat and protein components of churned cream. It is a semi-solid emulsion at room temperature, consisting of approximately 80% butterfat. It is used at room temperature as a spread, melted as a condiment, and used as an ingredient in baking, sauce making, pan frying, and other cooking procedures.There are other products that can be used instead. For instance: olive oil, ghee, greek yogurt, avocado, pumpkin puree or coconut oil.
Answer: olive oil, ghee, greek yogurt, avocado, pumpkin puree or coconut oil
Answer generated by BERT
Answer: olive oil , ghee , greek yogurt , avocado , pumpkin puree or coconut oil



In [31]:
# question 14 - potato
# source: https://en.wikipedia.org/wiki/Potato

question = "What can be used instead of potato?"
context = "The potato is a starchy tuber of the plant Solanum tuberosum and is a root vegetable native to the Americas. The plant is a perennial in the nightshade family Solanaceae.[2] Wild potato species can be found throughout the Americas, from Canada to southern Chile.[3] The potato was originally believed to have been domesticated by Native Americans independently in multiple locations,[4] but later genetic studies traced a single origin, in the area of present-day southern Peru and extreme northwestern Bolivia. Potatoes were domesticated there approximately 7,000–10,000 years ago, from a species in the Solanum brevicaule complex.[5][6][7] In the Andes region of South America, where the species is indigenous, some close relatives of the potato are cultivated."
answer = "cauliflower or cabbage"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of potato?
Context: The potato is a starchy tuber of the plant Solanum tuberosum and is a root vegetable native to the Americas. The plant is a perennial in the nightshade family Solanaceae.[2] Wild potato species can be found throughout the Americas, from Canada to southern Chile.[3] The potato was originally believed to have been domesticated by Native Americans independently in multiple locations,[4] but later genetic studies traced a single origin, in the area of present-day southern Peru and extreme northwestern Bolivia. Potatoes were domesticated there approximately 7,000–10,000 years ago, from a species in the Solanum brevicaule complex.[5][6][7] In the Andes region of South America, where the species is indigenous, some close relatives of the potato are cultivated.There are other products that can be used instead. For instance: cauliflower or cabbage.
Answer: cauliflower or cabbage
Answer generated by BERT
Answer: cauliflower or cabbage



In [32]:
# question 15 - frozen peas
# source: https://en.wikipedia.org/wiki/Frozen_Peas

question = "What can be used instead of frozen peas?"
context = "Frozen Peas is the colloquial term for a blooper audio clip in which American filmmaker Orson Welles performs narration for a series of British television advertisements for Findus. The clip is known informally as In July, or Yes, Always, based on several of Welles's complaints during the recording."
answer = "lima beans, green pepper, green onion or broad beans"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of frozen peas?
Context: Frozen Peas is the colloquial term for a blooper audio clip in which American filmmaker Orson Welles performs narration for a series of British television advertisements for Findus. The clip is known informally as In July, or Yes, Always, based on several of Welles's complaints during the recording.There are other products that can be used instead. For instance: lima beans, green pepper, green onion or broad beans.
Answer: lima beans, green pepper, green onion or broad beans
Answer generated by BERT
Answer: lima beans , green pepper , green onion or broad beans



In [33]:
# question 16 - mushrooms
# source: https://en.wikipedia.org/wiki/Mushroom

question = "What can be used instead of mushrooms?"
context = "A mushroom or toadstool is the fleshy, spore-bearing fruiting body of a fungus, typically produced above ground, on soil, or on its food source. The standard for the name mushroom is the cultivated white button mushroom, Agaricus bisporus; hence the word mushroom is most often applied to those fungi (Basidiomycota, Agaricomycetes) that have a stem (stipe), a cap (pileus), and gills (lamellae, sing. lamella) on the underside of the cap. Mushroom also describes a variety of other gilled fungi, with or without stems, therefore the term is used to describe the fleshy fruiting bodies of some Ascomycota. These gills produce microscopic spores that help the fungus spread across the ground or its occupant surface."
answer = "zucchini, eggplant, chickpeas, tofu, onions or cilantro"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of mushrooms?
Context: A mushroom or toadstool is the fleshy, spore-bearing fruiting body of a fungus, typically produced above ground, on soil, or on its food source. The standard for the name mushroom is the cultivated white button mushroom, Agaricus bisporus; hence the word mushroom is most often applied to those fungi (Basidiomycota, Agaricomycetes) that have a stem (stipe), a cap (pileus), and gills (lamellae, sing. lamella) on the underside of the cap. Mushroom also describes a variety of other gilled fungi, with or without stems, therefore the term is used to describe the fleshy fruiting bodies of some Ascomycota. These gills produce microscopic spores that help the fungus spread across the ground or its occupant surface.There are other products that can be used instead. For instance: zucchini, eggplant, chickpeas, tofu, onions or cilantro.
Answer: zucchini, eggplant, chickpeas, tofu, onions or cilantro
Answer generated by BERT
Answer: other pr

In [34]:
# question 17 - oat meal
# source: https://en.wikipedia.org/wiki/Mushroom

question = "What can be used instead of oat meal?"
context = "The oat (Avena sativa), sometimes called the common oat, is a species of cereal grain grown for its seed, which is known by the same name (usually in the plural, unlike other cereals and pseudocereals). While oats are suitable for human consumption as oatmeal and rolled oats, one of the most common uses is as livestock feed."
answer = "amaranth, teff, buckwheat, wheat berries, spelt or quinoa"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of oat meal?
Context: The oat (Avena sativa), sometimes called the common oat, is a species of cereal grain grown for its seed, which is known by the same name (usually in the plural, unlike other cereals and pseudocereals). While oats are suitable for human consumption as oatmeal and rolled oats, one of the most common uses is as livestock feed.There are other products that can be used instead. For instance: amaranth, teff, buckwheat, wheat berries, spelt or quinoa.
Answer: amaranth, teff, buckwheat, wheat berries, spelt or quinoa
Answer generated by BERT
Answer: amaranth , teff , buckwheat , wheat berries , spelt or quinoa



In [35]:
# question 18 - yeast
# source: https://en.wikipedia.org/wiki/Yeast

question = "What can be used instead of yeast?"
context = "Yeasts are eukaryotic, single-celled microorganisms classified as members of the fungus kingdom. The first yeast originated hundreds of millions of years ago, and at least 1,500 species are currently recognized.[1][2][3] They are estimated to constitute 1% of all described fungal species.[4] Yeasts are unicellular organisms that evolved from multicellular ancestors,[5] with some species having the ability to develop multicellular characteristics by forming strings of connected budding cells known as pseudohyphae or false hyphae.[6] Yeast sizes vary greatly, depending on species and environment, typically measuring 3–4 µm in diameter, although some yeasts can grow to 40 µm in size.[7] Most yeasts reproduce asexually by mitosis, and many do so by the asymmetric division process known as budding. With their single-celled growth habit, yeasts can be contrasted with molds, which grow hyphae. Fungal species that can take both forms (depending on temperature or other conditions) are called dimorphic fungi."
answer = "baking powder. baking soda or sourdough"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of yeast?
Context: Yeasts are eukaryotic, single-celled microorganisms classified as members of the fungus kingdom. The first yeast originated hundreds of millions of years ago, and at least 1,500 species are currently recognized.[1][2][3] They are estimated to constitute 1% of all described fungal species.[4] Yeasts are unicellular organisms that evolved from multicellular ancestors,[5] with some species having the ability to develop multicellular characteristics by forming strings of connected budding cells known as pseudohyphae or false hyphae.[6] Yeast sizes vary greatly, depending on species and environment, typically measuring 3–4 µm in diameter, although some yeasts can grow to 40 µm in size.[7] Most yeasts reproduce asexually by mitosis, and many do so by the asymmetric division process known as budding. With their single-celled growth habit, yeasts can be contrasted with molds, which grow hyphae. Fungal species that can take both forms (depen

In [36]:
# question 19 - bacon
# source: https://en.wikipedia.org/wiki/Bacon

question = "What can be used instead of bacon?"
context = "Bacon is a type of salt-cured pork[1] made from various cuts, typically the belly or less fatty parts of the back. It is eaten as a side dish (particularly in breakfasts), used as a central ingredient (e.g., the bacon, lettuce, and tomato sandwich (BLT), or as a flavouring or accent (as in bacon crumbles in a salad). Bacon is also used for barding and larding roasts, especially game, including venison and pheasant, and may also be used to insulate or flavour roast joints by being layered onto the meat. The word is derived from the Proto-Germanic *bakkon, meaning back meat.[2]"
answer = "pork, chicken or turkey bacon"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of bacon?
Context: Bacon is a type of salt-cured pork[1] made from various cuts, typically the belly or less fatty parts of the back. It is eaten as a side dish (particularly in breakfasts), used as a central ingredient (e.g., the bacon, lettuce, and tomato sandwich (BLT), or as a flavouring or accent (as in bacon crumbles in a salad). Bacon is also used for barding and larding roasts, especially game, including venison and pheasant, and may also be used to insulate or flavour roast joints by being layered onto the meat. The word is derived from the Proto-Germanic *bakkon, meaning back meat.[2]There are other products that can be used instead. For instance: pork, chicken or turkey bacon.
Answer: pork, chicken or turkey bacon
Answer generated by BERT
Answer: other products that can be used instead . for instance : pork , chicken or turkey bacon



In [37]:
# question 20 - tomatoes
# source: https://en.wikipedia.org/wiki/Tomato

question = "What can be used instead of tomatoes?"
context = "The tomato is the edible berry of the plant Solanum lycopersicum,[1][2] commonly known as a tomato plant. The species originated in western South America and Central America.[2][3] The Mexican Nahuatl word tomatl gave rise to the Spanish word tomate, from which the English word tomato derived.[3][4] Its domestication and use as a cultivated food may have originated with the indigenous peoples of Mexico.[2][5] The Aztecs used tomatoes in their cooking at the time of the Spanish conquest of the Aztec Empire, and after the Spanish encountered the tomato for the first time after their contact with the Aztecs, they brought the plant to Europe, in a widespread transfer of plants known as the Columbian exchange. From there, the tomato was introduced to other parts of the European-colonized world during the 16th century.[2]"
answer = "olives, red peppers or red capsicum"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of tomatoes?
Context: The tomato is the edible berry of the plant Solanum lycopersicum,[1][2] commonly known as a tomato plant. The species originated in western South America and Central America.[2][3] The Mexican Nahuatl word tomatl gave rise to the Spanish word tomate, from which the English word tomato derived.[3][4] Its domestication and use as a cultivated food may have originated with the indigenous peoples of Mexico.[2][5] The Aztecs used tomatoes in their cooking at the time of the Spanish conquest of the Aztec Empire, and after the Spanish encountered the tomato for the first time after their contact with the Aztecs, they brought the plant to Europe, in a widespread transfer of plants known as the Columbian exchange. From there, the tomato was introduced to other parts of the European-colonized world during the 16th century.[2]There are other products that can be used instead. For instance: olives, red peppers or red capsicum.
Answer: olives

In [38]:
# question 21 - baked beans
# source: https://en.wikipedia.org/wiki/Baked_beans

question = "What can be used instead of baked beans?"
context = "Baked beans is a dish traditionally containing white beans that are parboiled and then, in the US, baked in sauce at low temperature for a lengthy period.[1] In the United Kingdom, the dish is sometimes baked, but usually stewed in sauce.[2] Canned baked beans are not baked, but are cooked through a steam process.[3]"
answer = "peas or chickpeas"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What can be used instead of baked beans?
Context: Baked beans is a dish traditionally containing white beans that are parboiled and then, in the US, baked in sauce at low temperature for a lengthy period.[1] In the United Kingdom, the dish is sometimes baked, but usually stewed in sauce.[2] Canned baked beans are not baked, but are cooked through a steam process.[3]There are other products that can be used instead. For instance: peas or chickpeas.
Answer: peas or chickpeas
Answer generated by BERT
Answer: peas or chickpeas



In [39]:
# question 22 - paprika
# source: https://en.wikipedia.org/wiki/Paprika

question = "What can be used instead of paprika?"
context = "Paprika (American English more commonly /pəˈpriːkə/ (audio speaker iconlisten),[1] British English more commonly /ˈpæprɪkə/ (audio speaker iconlisten)[2]) is a spice made from dried and ground red peppers.[3] It is traditionally made from Capsicum annuum varietals in the Longum group, which also includes chili peppers, but the peppers used for paprika tend to be milder and have thinner flesh.[4][5] In some languages, but not English, the word paprika also refers to the plant and the fruit from which the spice is made, as well as to peppers in the Grossum group (e.g. bell peppers).[6][7]: 5, 73 "
answer = "catenne pepper, chili powder or chili flakes"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What can be used instead of paprika?
Context: Paprika (American English more commonly /pəˈpriːkə/ (audio speaker iconlisten),[1] British English more commonly /ˈpæprɪkə/ (audio speaker iconlisten)[2]) is a spice made from dried and ground red peppers.[3] It is traditionally made from Capsicum annuum varietals in the Longum group, which also includes chili peppers, but the peppers used for paprika tend to be milder and have thinner flesh.[4][5] In some languages, but not English, the word paprika also refers to the plant and the fruit from which the spice is made, as well as to peppers in the Grossum group (e.g. bell peppers).[6][7]: 5, 73 There are other products that can be used instead. For instance: catenne pepper, chili powder or chili flakes.
Answer: catenne pepper, chili powder or chili flakes
Answer generated by BERT
Answer: other products that can be used instead . for instance : catenne pepper , chili powder or chili flakes



In [40]:
# question 23 - feta
# source: https://en.wikipedia.org/wiki/Feta

question = "What can be used instead of feta?"
context = "Feta (Greek: φέτα, féta) is a Greek brined curd white cheese made from sheep's milk or from a mixture of sheep and goat's milk. It is soft, with small or no holes, a compact touch, few cuts, and no skin. It is formed into large blocks, and aged in brine. Its flavor is tangy and salty, ranging from mild to sharp. It is crumbly and has a slightly grainy texture. Feta is used as a table cheese, in salads such as Greek salad, and in pastries, notably the phyllo-based Greek dishes spanakopita (spinach pie) and tyropita (cheese pie). It is often served with olive oil or olives, and sprinkled with aromatic herbs such as oregano. It can also be served cooked (often grilled), as part of a sandwich, in omelettes, and many other dishes."
answer = "ricotta, halloumi, goats cheese or queso fresc"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What can be used instead of feta?
Context: Feta (Greek: φέτα, féta) is a Greek brined curd white cheese made from sheep's milk or from a mixture of sheep and goat's milk. It is soft, with small or no holes, a compact touch, few cuts, and no skin. It is formed into large blocks, and aged in brine. Its flavor is tangy and salty, ranging from mild to sharp. It is crumbly and has a slightly grainy texture. Feta is used as a table cheese, in salads such as Greek salad, and in pastries, notably the phyllo-based Greek dishes spanakopita (spinach pie) and tyropita (cheese pie). It is often served with olive oil or olives, and sprinkled with aromatic herbs such as oregano. It can also be served cooked (often grilled), as part of a sandwich, in omelettes, and many other dishes.There are other products that can be used instead. For instance: ricotta, halloumi, goats cheese or queso fresc.
Answer: ricotta, halloumi, goats cheese or queso fresc
Answer generated by BERT
Answer: other produ

In [41]:
# question 24 - parsley
# source: https://en.wikipedia.org/wiki/Parsley

question = "What can be used instead of parsley?"
context = "Parsley, or garden parsley (Petroselinum crispum) is a species of flowering plant in the family Apiaceae that is native to the central and eastern Mediterranean region (Sardinia, Lebanon, Israel, Cyprus, Turkey, southern Italy, Greece, Portugal, Spain, Malta, Morocco, Algeria, and Tunisia), but has been naturalized elsewhere in Europe, and is widely cultivated as a herb, and a vegetable."
answer = "chervil, tarragon, chives, oregano, arugula, endive, cilantro or basil"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What can be used instead of parsley?
Context: Parsley, or garden parsley (Petroselinum crispum) is a species of flowering plant in the family Apiaceae that is native to the central and eastern Mediterranean region (Sardinia, Lebanon, Israel, Cyprus, Turkey, southern Italy, Greece, Portugal, Spain, Malta, Morocco, Algeria, and Tunisia), but has been naturalized elsewhere in Europe, and is widely cultivated as a herb, and a vegetable.There are other products that can be used instead. For instance: chervil, tarragon, chives, oregano, arugula, endive, cilantro or basil.
Answer: chervil, tarragon, chives, oregano, arugula, endive, cilantro or basil
Answer generated by BERT
Answer: chervil , tarragon , chives , oregano , arugula , endive , cilantro or basil



In [42]:
# question 25 - cheddar
# source: https://en.wikipedia.org/wiki/Cheddar_cheese

question = "What can be used instead of cheddar?"
context = "Cheddar cheese (or simply cheddar) is a relatively hard, off-white (or orange if colourings such as annatto are added), sometimes sharp-tasting, natural cheese. Cheddar originates from the English village of Cheddar in Somerset.[1]"
answer = "cantal, edam, mimolette, gouda, emmental or french gruyere"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What can be used instead of cheddar?
Context: Cheddar cheese (or simply cheddar) is a relatively hard, off-white (or orange if colourings such as annatto are added), sometimes sharp-tasting, natural cheese. Cheddar originates from the English village of Cheddar in Somerset.[1]There are other products that can be used instead. For instance: cantal, edam, mimolette, gouda, emmental or french gruyere.
Answer: cantal, edam, mimolette, gouda, emmental or french gruyere
Answer generated by BERT
Answer: other products that can be used instead . for instance : cantal , edam , mimolette , gouda , emmental or french gruyere



In [43]:
# question 26 - pork fillet
# source: https://en.wikipedia.org/wiki/Pork_tenderloin

question = "What can be used instead of pork fillet?"
context = "Pork tenderloin, also called pork fillet, pork steak[1] or Gentleman's Cut, is a long thin cut of pork. As with all quadrupeds, the tenderloin refers to the psoas major muscle along the central spine portion, ventral to the lumbar vertebrae. This is the most tender part of the animal, because these muscles are used for posture, rather than locomotion."
answer = "pork sirloin, pork tenderloin, chicken, turkey or beef"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What can be used instead of pork fillet?
Context: Pork tenderloin, also called pork fillet, pork steak[1] or Gentleman's Cut, is a long thin cut of pork. As with all quadrupeds, the tenderloin refers to the psoas major muscle along the central spine portion, ventral to the lumbar vertebrae. This is the most tender part of the animal, because these muscles are used for posture, rather than locomotion.There are other products that can be used instead. For instance: pork sirloin, pork tenderloin, chicken, turkey or beef.
Answer: pork sirloin, pork tenderloin, chicken, turkey or beef
Answer generated by BERT
Answer: pork sirloin , pork tenderloin , chicken , turkey or beef



In [44]:
# question 27 - red peppers
# source: https://en.wikipedia.org/wiki/Red_Savina_pepper

question = "What can be used instead of red peppers?"
context = "The Red Savina pepper is a cultivar of the habanero chili (Capsicum chinense Jacquin), which has been selectively bred to produce spicier, heavier, and larger fruit, ultimately more potent than its derivative. Frank Garcia of GNS Spices, in Walnut, California, is credited as the developer of the Red Savina, but the exact methodology that Garcia used to select the hottest breeding strains is not publicly known."
answer = "poblano peppers, cubanelle, anaheim peppers, pepperoncini, jalapeno, pimiento, zucchini or broccoli"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What can be used instead of red peppers?
Context: The Red Savina pepper is a cultivar of the habanero chili (Capsicum chinense Jacquin), which has been selectively bred to produce spicier, heavier, and larger fruit, ultimately more potent than its derivative. Frank Garcia of GNS Spices, in Walnut, California, is credited as the developer of the Red Savina, but the exact methodology that Garcia used to select the hottest breeding strains is not publicly known.There are other products that can be used instead. For instance: poblano peppers, cubanelle, anaheim peppers, pepperoncini, jalapeno, pimiento, zucchini or broccoli.
Answer: poblano peppers, cubanelle, anaheim peppers, pepperoncini, jalapeno, pimiento, zucchini or broccoli
Answer generated by BERT
Answer: other products that can be used instead . for instance : poblano peppers , cubanelle , anaheim peppers , pepperoncini , jalapeno , pimiento , zucchini or broccoli



In [45]:
# question 28 - courgette
# source: https://en.wikipedia.org/wiki/Zucchini

question = "What can be used instead of courgette?"
context = "The name courgette is used in British, Hiberno-, Malaysian, New Zealand,[10][13] and South African English. It is loaned from French, where courgette (French pronunciation: ​[kuʁ.ʒɛt]) is a diminutive of courge, 'marrow'. Courgette is also used in Dutch."
answer = "cucumber or zucchini"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What can be used instead of courgette?
Context: The name courgette is used in British, Hiberno-, Malaysian, New Zealand,[10][13] and South African English. It is loaned from French, where courgette (French pronunciation: ​[kuʁ.ʒɛt]) is a diminutive of courge, 'marrow'. Courgette is also used in Dutch.There are other products that can be used instead. For instance: cucumber or zucchini.
Answer: cucumber or zucchini
Answer generated by BERT
Answer: cucumber or zucchini



In [46]:
# question 29 - chickpeas
# source: https://en.wikipedia.org/wiki/Chickpea

question = "What can be used instead of chickpea?"
context = "The chickpea or chick pea (Cicer arietinum) is an annual legume of the family Fabaceae, subfamily Faboideae.[2][3] Its different types are variously known as gram[4][5] or Bengal gram,[5] garbanzo[5] or garbanzo bean, or Egyptian pea.[4] Chickpea seeds are high in protein. It is one of the earliest cultivated legumes, and 9500-year-old remains have been found in the Middle East.[6][7]"
answer = "green peas, cannellini beans, black beans, lentils, soybeans, mung beans, peanuts or hazelnuts"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What can be used instead of chickpea?
Context: The chickpea or chick pea (Cicer arietinum) is an annual legume of the family Fabaceae, subfamily Faboideae.[2][3] Its different types are variously known as gram[4][5] or Bengal gram,[5] garbanzo[5] or garbanzo bean, or Egyptian pea.[4] Chickpea seeds are high in protein. It is one of the earliest cultivated legumes, and 9500-year-old remains have been found in the Middle East.[6][7]There are other products that can be used instead. For instance: green peas, cannellini beans, black beans, lentils, soybeans, mung beans, peanuts or hazelnuts.
Answer: green peas, cannellini beans, black beans, lentils, soybeans, mung beans, peanuts or hazelnuts
Answer generated by BERT
Answer: other products



In [47]:
# question 30 - chicken stock
# source: https://en.wikipedia.org/wiki/Stock_(food)

question = "What can be used instead of chicken stock?"
context = "Meat: Cooked meat still attached to bones is also used as an ingredient, especially with chicken stock. Meat cuts with a large amount of connective tissue, such as shoulder cuts, are also used."
answer = "beef stock or vegatable stock"
text = "There are other products that can be used instead. For instance: "
context = context + text + answer + "."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What can be used instead of chicken stock?
Context: Meat: Cooked meat still attached to bones is also used as an ingredient, especially with chicken stock. Meat cuts with a large amount of connective tissue, such as shoulder cuts, are also used.There are other products that can be used instead. For instance: beef stock or vegatable stock.
Answer: beef stock or vegatable stock
Answer generated by BERT
Answer: beef stock or vegatable stock




**Utensils Description**

In [48]:
# question 1 - bowl
# source: https://en.wikipedia.org/wiki/Bowl

question = "What is a bowl?"
context = "A bowl is a round dish or container typically used to prepare and serve food. The interior of a bowl is characteristically shaped like a spherical cap, with the edges and the bottom forming a seamless curve. This makes bowls especially suited for holding liquids and loose food, as the contents of the bowl are naturally concentrated in its center by the force of gravity. The exterior of a bowl is most often round but can be of any shape, including rectangular. The size of bowls varies from small bowls used to hold a single serving of food to large bowls, such as punch bowls or salad bowls, that are often used to hold or store more than one portion of food. There is some overlap between bowls, cups, and plates. Very small bowls, such as the tea bowl, are often called cups, while plates with especially deep wells are often called bowls."
answer = "a cooking container that is usually larger than a cup and kept in a cupboard in the kitchen, it is typically made from glass, ceramic, plastic."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a bowl?
Context: A bowl is a round dish or container typically used to prepare and serve food. The interior of a bowl is characteristically shaped like a spherical cap, with the edges and the bottom forming a seamless curve. This makes bowls especially suited for holding liquids and loose food, as the contents of the bowl are naturally concentrated in its center by the force of gravity. The exterior of a bowl is most often round but can be of any shape, including rectangular. The size of bowls varies from small bowls used to hold a single serving of food to large bowls, such as punch bowls or salad bowls, that are often used to hold or store more than one portion of food. There is some overlap between bowls, cups, and plates. Very small bowls, such as the tea bowl, are often called cups, while plates with especially deep wells are often called bowls.
Answer: a cooking container that is usually larger than a cup and kept in a cupboard in the kitchen, it is typically ma

In [49]:
# question 2 - spoon
# source: https://en.wikipedia.org/wiki/Spoon

question = "What is a spoon?"
context = "A spoon is a utensil consisting of a small shallow bowl (also known as a head), oval or round, at the end of a handle. A type of cutlery (sometimes called flatware in the United States), especially as part of a place setting, it is used primarily for transferring food to the mouth. Spoons are also used in food preparation to measure, mix, stir and toss ingredients and for serving food. Present day spoons are made from metal (notably flat silver or silverware, plated or solid), wood, porcelain or plastic. There are a wide variety of spoons that are made of a variety of materials and by different cultures for many different uses and foods."
answer = "an eating or cooking utensil consisting of a small shallow bowl with a relatively long handle made from metal, plastic, wood it is usually kept in the kitchen, in drawer."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a spoon?
Context: A spoon is a utensil consisting of a small shallow bowl (also known as a head), oval or round, at the end of a handle. A type of cutlery (sometimes called flatware in the United States), especially as part of a place setting, it is used primarily for transferring food to the mouth. Spoons are also used in food preparation to measure, mix, stir and toss ingredients and for serving food. Present day spoons are made from metal (notably flat silver or silverware, plated or solid), wood, porcelain or plastic. There are a wide variety of spoons that are made of a variety of materials and by different cultures for many different uses and foods.
Answer: an eating or cooking utensil consisting of a small shallow bowl with a relatively long handle made from metal, plastic, wood it is usually kept in the kitchen, in drawer.
Answer generated by BERT
Answer: a utensil



In [50]:
# question 3 - sieve
# source: https://en.wikipedia.org/wiki/Sieve

question = "What is a sieve?"
context = "A sieve, fine mesh strainer, or sift, is a device for separating wanted elements from unwanted material or for characterizing the particle size distribution of a sample, using a screen such as a woven mesh or net or perforated sheet material.[1] The word sift derives from sieve. In cooking, a sifter is used to separate and break up clumps in dry ingredients such as flour, as well as to aerate and combine them. A strainer (see Colander), meanwhile, is a form of sieve used to separate suspended solids from a liquid by filtration."
answer = "a metal or plastic device with meshes or perforations through which finer particles of a mixture are sifted, it is usually kept in a kitchen and stored in a cupboard. : "

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a sieve?
Context: A sieve, fine mesh strainer, or sift, is a device for separating wanted elements from unwanted material or for characterizing the particle size distribution of a sample, using a screen such as a woven mesh or net or perforated sheet material.[1] The word sift derives from sieve. In cooking, a sifter is used to separate and break up clumps in dry ingredients such as flour, as well as to aerate and combine them. A strainer (see Colander), meanwhile, is a form of sieve used to separate suspended solids from a liquid by filtration.
Answer: a metal or plastic device with meshes or perforations through which finer particles of a mixture are sifted, it is usually kept in a kitchen and stored in a cupboard. : 
Answer generated by BERT
Answer: a device for separating wanted elements from unwanted material



In [51]:
# question 4 - pan
# source: https://en.wikipedia.org/wiki/Sieve

question = "What is a pan?"
context = "A frying pan, frypan, or skillet is a flat-bottomed pan used for frying, searing, and browning foods. It is typically 20 to 30 cm (8 to 12 in) in diameter with relatively low sides that flare outwards, a long handle, and no lid. Larger pans may have a small grab handle opposite the main handle. A pan of similar dimensions, but with less flared, more vertical sides and often with a lid, is called a sauté pan. While a sauté pan can be used as a frying pan, it is designed for lower heat cooking methods, namely sautéing."
answer = "a shallow and open cooking container made from metal, it is usually found in a kitchen and stored in a cupboard."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a pan?
Context: A frying pan, frypan, or skillet is a flat-bottomed pan used for frying, searing, and browning foods. It is typically 20 to 30 cm (8 to 12 in) in diameter with relatively low sides that flare outwards, a long handle, and no lid. Larger pans may have a small grab handle opposite the main handle. A pan of similar dimensions, but with less flared, more vertical sides and often with a lid, is called a sauté pan. While a sauté pan can be used as a frying pan, it is designed for lower heat cooking methods, namely sautéing.
Answer: a shallow and open cooking container made from metal, it is usually found in a kitchen and stored in a cupboard.
Answer generated by BERT
Answer: a flat - bottomed pan



In [52]:
# question 5 - baking tray
# source: https://en.wikipedia.org/wiki/Sheet_pan

question = "What is a baking tray?"
context = "A baking tray is a flat, rectangular metal pan used in an oven. It is often used for baking bread rolls, pastries and flat products such as cookies, sheet cakes, Swiss rolls and pizzas. These pans, like all bakeware, can be made of a variety of materials, but are primarily aluminum or stainless steel. The most basic sheet pan is literally a sheet of metal. Commercial pans are sometimes made from aluminized steel which combines the conductive, reflective, and food adherence properties of aluminum, with the rigidity, mass, and strength of the inner steel core (in this process, the aluminum surface is typically 90% aluminum and 10% silicon, which is not quite the same as pure aluminum)."
answer = "a metal rectangular sheet with a rolled edge used for baking, it is usually found in a kitchen inside a cupboard or draw."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a baking tray?
Context: A baking tray is a flat, rectangular metal pan used in an oven. It is often used for baking bread rolls, pastries and flat products such as cookies, sheet cakes, Swiss rolls and pizzas. These pans, like all bakeware, can be made of a variety of materials, but are primarily aluminum or stainless steel. The most basic sheet pan is literally a sheet of metal. Commercial pans are sometimes made from aluminized steel which combines the conductive, reflective, and food adherence properties of aluminum, with the rigidity, mass, and strength of the inner steel core (in this process, the aluminum surface is typically 90% aluminum and 10% silicon, which is not quite the same as pure aluminum).
Answer: a metal rectangular sheet with a rolled edge used for baking, it is usually found in a kitchen inside a cupboard or draw.
Answer generated by BERT
Answer: a flat , rectangular metal pan used in an oven



In [53]:
# question 6 - whisk
# source: https://en.wikipedia.org/wiki/Whisk

question = "What is a whisk?"
context = "A whisk is a cooking utensil which can be used to blend ingredients smooth or to incorporate air into a mixture, in a process known as whisking or whipping. Most whisks consist of a long, narrow handle with a series of wire loops joined at the end. The loops can have different shapes depending on a whisk's intended functions. The wires are usually metal, but some are plastic for use with nonstick cookware. Whisks are also made from bamboo. Whisks are commonly used to whip egg whites into a firm foam to make meringue, or to whip cream into whipped cream."
answer = "a metal or plastic wire utensil used for beating food by hand and is usually found in a kitchen inside a drawer."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a whisk?
Context: A whisk is a cooking utensil which can be used to blend ingredients smooth or to incorporate air into a mixture, in a process known as whisking or whipping. Most whisks consist of a long, narrow handle with a series of wire loops joined at the end. The loops can have different shapes depending on a whisk's intended functions. The wires are usually metal, but some are plastic for use with nonstick cookware. Whisks are also made from bamboo. Whisks are commonly used to whip egg whites into a firm foam to make meringue, or to whip cream into whipped cream.
Answer: a metal or plastic wire utensil used for beating food by hand and is usually found in a kitchen inside a drawer.
Answer generated by BERT
Answer: a cooking utensil



In [54]:
# question 7 - oven
# source: https://en.wikipedia.org/wiki/Oven

question = "What is a oven?"
context = "An oven is a tool which is used to expose materials to a hot environment. Ovens contain a hollow chamber and provide a means of heating the chamber in a controlled way.[1] In use since antiquity, they have been used to accomplish a wide variety of tasks requiring controlled heating.[2] Because they are used for a variety of purposes, there are many different types of ovens. These types differ depending on their intended purpose and based upon how they generate heat."
answer = "a cooking chamber used for baking, heating, or drying and is usually found in a kitchen. "

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a oven?
Context: An oven is a tool which is used to expose materials to a hot environment. Ovens contain a hollow chamber and provide a means of heating the chamber in a controlled way.[1] In use since antiquity, they have been used to accomplish a wide variety of tasks requiring controlled heating.[2] Because they are used for a variety of purposes, there are many different types of ovens. These types differ depending on their intended purpose and based upon how they generate heat.
Answer: a cooking chamber used for baking, heating, or drying and is usually found in a kitchen. 
Answer generated by BERT
Answer: a tool which is used to expose materials to a hot environment



In [55]:
# question 8 - cellophane
# source: https://en.wikipedia.org/wiki/Cellophane

question = "What is a cellophane?"
context = "Cellophane is a thin, transparent sheet made of regenerated cellulose. Its low permeability to air, oils, greases, bacteria, and water makes it useful for food packaging. Cellophane is highly permeable to water vapour, but may be coated with nitrocellulose lacquer to prevent this. As well as food packaging, cellophane is used in transparent pressure-sensitive tape, tubing and many other similar applications. Cellophane is biosourced, compostable, and biodegradable.[1] Production uses carbon disulfide (CS2), which has been found to be highly toxic to workers; modern manufacturers do not give any information on worker harms.[2]"
answer = "thin transparent plastic sheets used to package or cover food items, it is usually kept in a kitchen inside a drawer or cupboard."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a cellophane?
Context: Cellophane is a thin, transparent sheet made of regenerated cellulose. Its low permeability to air, oils, greases, bacteria, and water makes it useful for food packaging. Cellophane is highly permeable to water vapour, but may be coated with nitrocellulose lacquer to prevent this. As well as food packaging, cellophane is used in transparent pressure-sensitive tape, tubing and many other similar applications. Cellophane is biosourced, compostable, and biodegradable.[1] Production uses carbon disulfide (CS2), which has been found to be highly toxic to workers; modern manufacturers do not give any information on worker harms.[2]
Answer: thin transparent plastic sheets used to package or cover food items, it is usually kept in a kitchen inside a drawer or cupboard.
Answer generated by BERT
Answer: a thin , transparent sheet made of regenerated cellulose



In [56]:
# question 9 - tea towel
# source: https://en.wikipedia.org/wiki/Towel#dishtowel

question = "What is a tea towel?"
context = "Tea towels or tea cloths (UK and Canadian English), called dishtowels or dish towels in America, originated in 18th-century England. They are absorbent towels made from soft, lint-free linen. They are used in the kitchen to dry dishes, cutlery, etc. after they are washed. The towels are also used during tea time. They can be wrapped around the tea pot to keep the tea warm, prevent drips, and keep one's hand from being burned by the hot tea pot handle when serving the tea. They are commonly made of cotton rather than linen; they are also used for drying glassware, but sometimes a special glass cloth is used for that purpose."
answer = "a fabric cloth used for drying dishes, it is typically found in a kitchen or airing cupboard."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a tea towel?
Context: Tea towels or tea cloths (UK and Canadian English), called dishtowels or dish towels in America, originated in 18th-century England. They are absorbent towels made from soft, lint-free linen. They are used in the kitchen to dry dishes, cutlery, etc. after they are washed. The towels are also used during tea time. They can be wrapped around the tea pot to keep the tea warm, prevent drips, and keep one's hand from being burned by the hot tea pot handle when serving the tea. They are commonly made of cotton rather than linen; they are also used for drying glassware, but sometimes a special glass cloth is used for that purpose.
Answer: a fabric cloth used for drying dishes, it is typically found in a kitchen or airing cupboard.
Answer generated by BERT
Answer: absorbent towels made from soft , lint - free linen



In [57]:
# question 10 - frying pan
# source: https://en.wikipedia.org/wiki/Frying_pan

question = "What is a frying pan?"
context = "A frying pan, frypan, or skillet is a flat-bottomed pan used for frying, searing, and browning foods. It is typically 20 to 30 cm (8 to 12 in) in diameter with relatively low sides that flare outwards, a long handle, and no lid. Larger pans may have a small grab handle opposite the main handle. A pan of similar dimensions, but with less flared, more vertical sides and often with a lid, is called a sauté pan. While a sauté pan can be used as a frying pan, it is designed for lower heat cooking methods, namely sautéing."
answer = "an open metal container with a handle that is used for frying foods, it is usually found in a kitchen inside a cupboard."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a frying pan?
Context: A frying pan, frypan, or skillet is a flat-bottomed pan used for frying, searing, and browning foods. It is typically 20 to 30 cm (8 to 12 in) in diameter with relatively low sides that flare outwards, a long handle, and no lid. Larger pans may have a small grab handle opposite the main handle. A pan of similar dimensions, but with less flared, more vertical sides and often with a lid, is called a sauté pan. While a sauté pan can be used as a frying pan, it is designed for lower heat cooking methods, namely sautéing.
Answer: an open metal container with a handle that is used for frying foods, it is usually found in a kitchen inside a cupboard.
Answer generated by BERT
Answer: a flat - bottomed pan



In [58]:
# question 11 - kettle
# source: https://en.wikipedia.org/wiki/Kettle

question = "What is a kettle?"
context = "A kettle, sometimes called a tea kettle or teakettle, is a type of pot specialized for boiling water, commonly with a lid, spout, and handle, or a small electric kitchen appliance of similar shape that functions in a self-contained manner. Kettles can be heated either by placing on a stove, or by their own internal electric heating element in the appliance versions. As indicated by its name, the kettle was and is often used as teaware to brew tea or prepare a tisane. Some very modern versions do more than just boil water, and also make the tea and keep it warm."
answer = "a metal or plastic vessel used for boiling liquids, it is usually found in a kitchen on a tabletop or inside a cupboard"

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a kettle?
Context: A kettle, sometimes called a tea kettle or teakettle, is a type of pot specialized for boiling water, commonly with a lid, spout, and handle, or a small electric kitchen appliance of similar shape that functions in a self-contained manner. Kettles can be heated either by placing on a stove, or by their own internal electric heating element in the appliance versions. As indicated by its name, the kettle was and is often used as teaware to brew tea or prepare a tisane. Some very modern versions do more than just boil water, and also make the tea and keep it warm.
Answer: a metal or plastic vessel used for boiling liquids, it is usually found in a kitchen on a tabletop or inside a cupboard
Answer generated by BERT
Answer: a type of pot specialized for boiling water



In [59]:
# question 12 - kitchen knife
# source: https://en.wikipedia.org/wiki/Kitchen_knife

question = "What is a kitchen knife?"
context = "A kitchen knife is any knife that is intended to be used in food preparation. While much of this work can be accomplished with a few general-purpose knives – notably a large chef's knife, a tough cleaver, a small paring knife and some sort of serrated blade (such as a bread knife or serrated utility knife) – there are also many specialized knives that are designed for specific tasks. Kitchen knives can be made from several different materials."
answer = "a cutting instrument consisting of a sharp blade fastened to a handle, it is usually found in a kitchen inside a draw."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a kitchen knife?
Context: A kitchen knife is any knife that is intended to be used in food preparation. While much of this work can be accomplished with a few general-purpose knives – notably a large chef's knife, a tough cleaver, a small paring knife and some sort of serrated blade (such as a bread knife or serrated utility knife) – there are also many specialized knives that are designed for specific tasks. Kitchen knives can be made from several different materials.
Answer: a cutting instrument consisting of a sharp blade fastened to a handle, it is usually found in a kitchen inside a draw.
Answer generated by BERT
Answer: any knife that is intended to be used in food preparation



In [60]:
# question 13 - saucepan
# source: https://en.wikipedia.org/wiki/Cookware_and_bakeware#Types_of_cookware_and_bakeware

question = "What is a saucepan?"
context = "Saucepans are round, vertical-walled vessels used for simmering or boiling. Saucepans generally have one long handle. Larger pans of similar shape with two ear handles are sometimes called sauce-pots or soup pots (3–12 litres). Saucepans and saucepots are denominated by volume (usually 1–8 L). While saucepots often resemble Dutch ovens in shape, they are generally lighter. Very small saucepans used for heating milk are referred to as milk pans - such saucepans usually have a lip for pouring heated milk."
answer = "a small deep metal cooking pan with a handle and is usually kept in a kitchen cupboard."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a saucepan?
Context: Saucepans are round, vertical-walled vessels used for simmering or boiling. Saucepans generally have one long handle. Larger pans of similar shape with two ear handles are sometimes called sauce-pots or soup pots (3–12 litres). Saucepans and saucepots are denominated by volume (usually 1–8 L). While saucepots often resemble Dutch ovens in shape, they are generally lighter. Very small saucepans used for heating milk are referred to as milk pans - such saucepans usually have a lip for pouring heated milk.
Answer: a small deep metal cooking pan with a handle and is usually kept in a kitchen cupboard.
Answer generated by BERT
Answer: round , vertical - walled vessels



In [61]:
# question 14 - greaseproof paper
# source: https://en.wikipedia.org/wiki/Greaseproof_paper

question = "What is a greaseproof paper?"
context = "Greaseproof paper is paper that is impermeable to oil or grease and is normally used in cooking or food packaging. Normally greaseproof paper is produced by refining the paper stock and thus create a sheet with very low porosity. This sheet is passed between hard pressure rollers (supercalendered) to further increase the density, creating a paper called glassine. The glassine is treated with starches, alginates or carboxymethyl cellulose (CMC) in a size press to fill pores or treat the paper chemically to make it fat repellent. Basis weights are usually 30–50 g/m2.[1][2]"
answer = "a heavy stiff waxed paper used in cooking and baking to prevent food from sticking to cooking trays and is usually kept inside a kitchen drawer."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a greaseproof paper?
Context: Greaseproof paper is paper that is impermeable to oil or grease and is normally used in cooking or food packaging. Normally greaseproof paper is produced by refining the paper stock and thus create a sheet with very low porosity. This sheet is passed between hard pressure rollers (supercalendered) to further increase the density, creating a paper called glassine. The glassine is treated with starches, alginates or carboxymethyl cellulose (CMC) in a size press to fill pores or treat the paper chemically to make it fat repellent. Basis weights are usually 30–50 g/m2.[1][2]
Answer: a heavy stiff waxed paper used in cooking and baking to prevent food from sticking to cooking trays and is usually kept inside a kitchen drawer.
Answer generated by BERT
Answer: paper that is impermeable to oil or grease



In [62]:
# question 15 - cup
# source: https://en.wikipedia.org/wiki/Cup

question = "What is a cup?"
context = "A cup is an open-top container used to hold or cold liquids for pouring or drinking; while mainly used for drinking, it also can be used to store solids for pouring (e.g., sugar, flour, grains, salt).[1][2] Cups may be made of glass, metal, china,[3] clay, wood, stone, polystyrene, plastic, aluminium or other materials, and are usually fixed with a stem, handles, or other adornments. Cups are used for quenching thirst across a wide range of cultures and social classes,[4] and different styles of cups may be used for different liquids or in different situations.[5] Cups of different styles may be used for different types of liquids or other foodstuffs (e.g. teacups and measuring cups), in different situations (e.g. at water stations or in ceremonies and rituals),or for decoration.[5][6]"
answer = "a ceramic, porcelain or glass bowl-shaped drinking vessel usually kept in a kitchen cupboard."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a cup?
Context: A cup is an open-top container used to hold or cold liquids for pouring or drinking; while mainly used for drinking, it also can be used to store solids for pouring (e.g., sugar, flour, grains, salt).[1][2] Cups may be made of glass, metal, china,[3] clay, wood, stone, polystyrene, plastic, aluminium or other materials, and are usually fixed with a stem, handles, or other adornments. Cups are used for quenching thirst across a wide range of cultures and social classes,[4] and different styles of cups may be used for different liquids or in different situations.[5] Cups of different styles may be used for different types of liquids or other foodstuffs (e.g. teacups and measuring cups), in different situations (e.g. at water stations or in ceremonies and rituals),or for decoration.[5][6]
Answer: a ceramic, porcelain or glass bowl-shaped drinking vessel usually kept in a kitchen cupboard.
Answer generated by BERT
Answer: an open - top container used to ho

In [63]:
# question 16 - measuring jug
# source: https://en.wikipedia.org/wiki/Measuring_cup

question = "What is a measuring jug?"
context = "A measuring cup is a kitchen utensil used primarily to measure the volume of liquid or bulk solid cooking ingredients such as flour and sugar, especially for volumes from about 50 mL (2 fl oz) upwards. Measuring cups are also used to measure washing powder, liquid detergents and bleach for clothes washing. The cup will usually have a scale marked in cups and fractions of a cup, and often with fluid measure and weight of a selection of dry foodstuffs. Measuring cups may be made of plastic, glass, or metal. Transparent (or translucent) cups can be read from an external scale; metal ones only from a dipstick or scale marked on the inside."
answer = "a plastic or glass cup that has markings for measuring liquids when cooking, it is usually kept inside a kitchen cupboard."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a measuring jug?
Context: A measuring cup is a kitchen utensil used primarily to measure the volume of liquid or bulk solid cooking ingredients such as flour and sugar, especially for volumes from about 50 mL (2 fl oz) upwards. Measuring cups are also used to measure washing powder, liquid detergents and bleach for clothes washing. The cup will usually have a scale marked in cups and fractions of a cup, and often with fluid measure and weight of a selection of dry foodstuffs. Measuring cups may be made of plastic, glass, or metal. Transparent (or translucent) cups can be read from an external scale; metal ones only from a dipstick or scale marked on the inside.
Answer: a plastic or glass cup that has markings for measuring liquids when cooking, it is usually kept inside a kitchen cupboard.
Answer generated by BERT
Answer: a kitchen utensil



In [64]:
# question 17 - dinner plate
# source: https://en.wikipedia.org/wiki/Plate_(dishware)

question = "What is a dinner plate?"
context = "A plate is a broad, concave, but mainly flat vessel on which food can be served.[1] A plate can also be used for ceremonial or decorative purposes. Most plates are circular, but they may be any shape, or made of any water-resistant material. Generally plates are raised round the edges, either by a curving up, or a wider lip or raised portion. Vessels with no lip, especially if they have a more rounded profile, are likely to be considered as bowls or dishes, as are very large vessels with a plate shape. Plates are dishware, and tableware. Plates in wood, pottery and metal go back into antiquity in many cultures."
answer = "a ceramic large plate usually 10 inches in diameter used for the main course of a meal, it is usually kept in a kitchen cupboard. "

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a dinner plate?
Context: A plate is a broad, concave, but mainly flat vessel on which food can be served.[1] A plate can also be used for ceremonial or decorative purposes. Most plates are circular, but they may be any shape, or made of any water-resistant material. Generally plates are raised round the edges, either by a curving up, or a wider lip or raised portion. Vessels with no lip, especially if they have a more rounded profile, are likely to be considered as bowls or dishes, as are very large vessels with a plate shape. Plates are dishware, and tableware. Plates in wood, pottery and metal go back into antiquity in many cultures.
Answer: a ceramic large plate usually 10 inches in diameter used for the main course of a meal, it is usually kept in a kitchen cupboard. 
Answer generated by BERT
Answer: a broad , concave , but mainly flat vessel on which food can be served



In [65]:
# question 18 - rolling pin
# source: https://en.wikipedia.org/wiki/Rolling_pin

question = "What is a rolling pin?"
context = "A rolling pin is a cylindrical food preparation utensil used to shape and flatten dough. Two styles of rolling pin are found: rollers and rods. Roller types consists of a thick cylinder with small handles at each end; rod type rolling pins are usually thin tapered batons. Rolling pins of different styles and materials offer varying advantages, as they are used for different tasks in cooking and baking."
answer = "a long wooden or plastic cylinder for rolling out dough, it is usually kept in a kitchen drawer."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a rolling pin?
Context: A rolling pin is a cylindrical food preparation utensil used to shape and flatten dough. Two styles of rolling pin are found: rollers and rods. Roller types consists of a thick cylinder with small handles at each end; rod type rolling pins are usually thin tapered batons. Rolling pins of different styles and materials offer varying advantages, as they are used for different tasks in cooking and baking.
Answer: a long wooden or plastic cylinder for rolling out dough, it is usually kept in a kitchen drawer.
Answer generated by BERT
Answer: a cylindrical food preparation utensil



In [66]:
# question 19 - food processor
# source: https://en.wikipedia.org/wiki/Food_processor

question = "What is a food processor?"
context = "A food processor is a kitchen appliance used to facilitate repetitive tasks in the preparation of food. Today, the term almost always refers to an electric-motor-driven appliance, although there are some manual devices also referred to as food processors. Food processors are similar to blenders in many forms. A food processor typically requires little to no liquid during use, unlike a blender, which requires a set amount of liquid in order for the blade to properly blend the food. Food processors are used to blend, chop, dice, and slice, allowing for quicker meal preparation.[1]"
answer = "an electric kitchen appliance with a set of interchangeable blades revolving inside a container, it is usually kept in a kitchen cupboard."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a food processor?
Context: A food processor is a kitchen appliance used to facilitate repetitive tasks in the preparation of food. Today, the term almost always refers to an electric-motor-driven appliance, although there are some manual devices also referred to as food processors. Food processors are similar to blenders in many forms. A food processor typically requires little to no liquid during use, unlike a blender, which requires a set amount of liquid in order for the blade to properly blend the food. Food processors are used to blend, chop, dice, and slice, allowing for quicker meal preparation.[1]
Answer: an electric kitchen appliance with a set of interchangeable blades revolving inside a container, it is usually kept in a kitchen cupboard.
Answer generated by BERT
Answer: a kitchen appliance



In [67]:
# question 20 - dinner fork
# source: https://en.wikipedia.org/wiki/Fork

question = "What is a dinner fork?"
context = "In cutlery or kitchenware, a fork (from Latin: furca 'pitchfork') is a utensil, now usually made of metal, whose long handle terminates in a head that branches into several narrow and often slightly curved tines with which one can spear foods either to hold them to cut with a knife or to lift them to the mouth."
answer = "a large metal fork with 3 or 4 times, it is usually found in a kitchen drawer."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a dinner fork?
Context: In cutlery or kitchenware, a fork (from Latin: furca 'pitchfork') is a utensil, now usually made of metal, whose long handle terminates in a head that branches into several narrow and often slightly curved tines with which one can spear foods either to hold them to cut with a knife or to lift them to the mouth.
Answer: a large metal fork with 3 or 4 times, it is usually found in a kitchen drawer.
Answer generated by BERT
Answer: a utensil



In [68]:
# question 21 - pastry brush
# source: https://en.wikipedia.org/wiki/Pastry_brush

question = "What is a pastry brush?"
context = "A pastry brush, also known as a basting brush, is a cooking utensil used to spread butter, oil or glaze on food. Traditional pastry brushes are made with natural bristles or a plastic or nylon fiber similar to a paint brush, while modern kitchen brushes may have silicone bristles. In baking breads and pastries, a pastry brush is used to spread a glaze or egg wash on the crust or surface of the food.[1] In roasting meats, a pastry brush may be used to sop up juices or drippings from under pan and spread them on the surface of the meat to crisp the skin."
answer = "also known as a basting brush, it is a wooden or plastic cooking utensil used to spread butter, oil or glaze on food, it is usually kept in a kitchen  drawer."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")


Question: What is a pastry brush?
Context: A pastry brush, also known as a basting brush, is a cooking utensil used to spread butter, oil or glaze on food. Traditional pastry brushes are made with natural bristles or a plastic or nylon fiber similar to a paint brush, while modern kitchen brushes may have silicone bristles. In baking breads and pastries, a pastry brush is used to spread a glaze or egg wash on the crust or surface of the food.[1] In roasting meats, a pastry brush may be used to sop up juices or drippings from under pan and spread them on the surface of the meat to crisp the skin.
Answer: also known as a basting brush, it is a wooden or plastic cooking utensil used to spread butter, oil or glaze on food, it is usually kept in a kitchen  drawer.
Answer generated by BERT
Answer: a cooking utensil



In [69]:
# question 22 - fridge
# source: https://en.wikipedia.org/wiki/Refrigerator

question = "What is a fridge?"
context = "A refrigerator (colloquially fridge) is a commercial and home appliance consisting of a thermally insulated compartment and a heat pump (mechanical, electronic or chemical) that transfers heat from its inside to its external environment so that its inside is cooled to a temperature below the room temperature. The refrigerator should be kept at or below 4 °C (40 °F) and the freezer should be regulated at −18 °C (0 °F).[according to whom?] Refrigeration is an essential food storage technique around the world. "
answer = "an appliance for keeping food or other items cool, it is usually found in a kitchen or utility room."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What is a fridge?
Context: A refrigerator (colloquially fridge) is a commercial and home appliance consisting of a thermally insulated compartment and a heat pump (mechanical, electronic or chemical) that transfers heat from its inside to its external environment so that its inside is cooled to a temperature below the room temperature. The refrigerator should be kept at or below 4 °C (40 °F) and the freezer should be regulated at −18 °C (0 °F).[according to whom?] Refrigeration is an essential food storage technique around the world. 
Answer: an appliance for keeping food or other items cool, it is usually found in a kitchen or utility room.
Answer generated by BERT
Answer: a commercial and home appliance



In [70]:
# question 23 - casserole dish
# source: https://en.wikipedia.org/wiki/Dutch_oven

question = "What is a casserole dish?"
context = "A Dutch oven (not to be confused with masonry oven) is a thick-walled cooking pot with a tight-fitting lid. Dutch ovens are usually made of seasoned cast iron; however, some Dutch ovens are instead made of cast aluminium, or ceramic. Some metal varieties are enameled rather than being seasoned, and these are sometimes called French ovens. Dutch ovens have been used as cooking vessels for hundreds of years. They are called casserole dishes in English-speaking countries other than the United States (casserole means pan in French), and cocottes in French.[citation needed] They are similar to both the Japanese tetsunabe and the sač, a traditional Balkan cast-iron oven, and are related to the South African potjie, the Australian Bedourie oven and Spanish cazuela."
answer = "a large and deep, glass or ceramic cooking bowl, it is usually kept in a kitchen cupboard."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What is a casserole dish?
Context: A Dutch oven (not to be confused with masonry oven) is a thick-walled cooking pot with a tight-fitting lid. Dutch ovens are usually made of seasoned cast iron; however, some Dutch ovens are instead made of cast aluminium, or ceramic. Some metal varieties are enameled rather than being seasoned, and these are sometimes called French ovens. Dutch ovens have been used as cooking vessels for hundreds of years. They are called casserole dishes in English-speaking countries other than the United States (casserole means pan in French), and cocottes in French.[citation needed] They are similar to both the Japanese tetsunabe and the sač, a traditional Balkan cast-iron oven, and are related to the South African potjie, the Australian Bedourie oven and Spanish cazuela.
Answer: a large and deep, glass or ceramic cooking bowl, it is usually kept in a kitchen cupboard.
Answer generated by BERT
Answer: pan



In [71]:
# question 24 - teaspoon
# source: https://en.wikipedia.org/wiki/Teaspoon

question = "What is a teaspoon?"
context = "A teaspoon (tsp.) is an item of cutlery. It is a small spoon that can be used to stir a cup of tea or coffee, or as a tool for measuring volume.[1][2] The size of teaspoons ranges from about 2.5 to 7.3 mL (0.088 to 0.257 imp fl oz; 0.085 to 0.247 US fl oz). For cooking purposes and, more importantly, for dosing of medicine, a teaspoonful is defined as 5 mL (0.18 imp fl oz; 0.17 US fl oz), and standard measuring spoons are used.[3]"
answer = "a small spoon that is used especially for eating soft foods and stirring beverages and that holds about one third of a tablespoon, it is usually stored in a kitchen drawer."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What is a teaspoon?
Context: A teaspoon (tsp.) is an item of cutlery. It is a small spoon that can be used to stir a cup of tea or coffee, or as a tool for measuring volume.[1][2] The size of teaspoons ranges from about 2.5 to 7.3 mL (0.088 to 0.257 imp fl oz; 0.085 to 0.247 US fl oz). For cooking purposes and, more importantly, for dosing of medicine, a teaspoonful is defined as 5 mL (0.18 imp fl oz; 0.17 US fl oz), and standard measuring spoons are used.[3]
Answer: a small spoon that is used especially for eating soft foods and stirring beverages and that holds about one third of a tablespoon, it is usually stored in a kitchen drawer.
Answer generated by BERT
Answer: a small spoon



In [72]:
# question 25 - wooden spoon
# source: https://en.wikipedia.org/wiki/Wooden_spoon

question = "What is a wooden spoon?"
context = "A wooden spoon is a utensil commonly used in food preparation.The word spoon derives from an ancient word meaning a chip of wood or horn carved from a larger piece.[1] Wooden spoons were easy to carve and thus inexpensive, making them common throughout history."
answer = "a large wooden spoon for stirring ingredients, it is usually stored in a kitchen drawer."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What is a wooden spoon?
Context: A wooden spoon is a utensil commonly used in food preparation.The word spoon derives from an ancient word meaning a chip of wood or horn carved from a larger piece.[1] Wooden spoons were easy to carve and thus inexpensive, making them common throughout history.
Answer: a large wooden spoon for stirring ingredients, it is usually stored in a kitchen drawer.
Answer generated by BERT
Answer: a utensil commonly used in food preparation



In [73]:
# question 26 - chopsticks
# source: https://en.wikipedia.org/wiki/Chopsticks

question = "What is a chopsticks?"
context = "Chopsticks are shaped pairs of equal-length sticks of Chinese origin that have been used as kitchen and eating utensils in most of East Asia for over three millennia. They are held in the dominant hand, secured by fingers, and wielded as extensions of the hand, to pick up food."
answer = "a pair of slender plastic or wooden sticks held between thumb and fingers and used chiefly in Asian countries to lift food to the mouth, they are usually stored in a kitchen drawer."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What is a chopsticks?
Context: Chopsticks are shaped pairs of equal-length sticks of Chinese origin that have been used as kitchen and eating utensils in most of East Asia for over three millennia. They are held in the dominant hand, secured by fingers, and wielded as extensions of the hand, to pick up food.
Answer: a pair of slender plastic or wooden sticks held between thumb and fingers and used chiefly in Asian countries to lift food to the mouth, they are usually stored in a kitchen drawer.
Answer generated by BERT
Answer: shaped pairs of equal - length sticks of chinese origin



In [74]:
# question 27 - colander
# source: https://en.wikipedia.org/wiki/Colander

question = "What is a colander?"
context = "A colander (or cullender) is a kitchen utensil used to strain foods such as pasta or to rinse vegetables.[1] The perforated nature of the colander allows liquid to drain through while retaining the solids inside. It is sometimes also called a pasta strainer or kitchen sieve. Traditionally, colanders are made of a light metal, such as aluminium or thinly rolled stainless steel. Colanders are also made of plastic, silicone, ceramic, and enamelware.[2]"
answer = "a perforated plastic or metal cylinder for washing or draining food, it is usually stored in a kitchen cupboard."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What is a colander?
Context: A colander (or cullender) is a kitchen utensil used to strain foods such as pasta or to rinse vegetables.[1] The perforated nature of the colander allows liquid to drain through while retaining the solids inside. It is sometimes also called a pasta strainer or kitchen sieve. Traditionally, colanders are made of a light metal, such as aluminium or thinly rolled stainless steel. Colanders are also made of plastic, silicone, ceramic, and enamelware.[2]
Answer: a perforated plastic or metal cylinder for washing or draining food, it is usually stored in a kitchen cupboard.
Answer generated by BERT
Answer: a kitchen utensil



In [75]:
# question 28 - filleting knife
# source: https://en.wikipedia.org/wiki/Fillet_knife

question = "What is a fillet knife?"
context = "A fillet knife (also called a filleting knife) is a kitchen knife used for filleting. It gives good control and aids in filleting. It is a very flexible member of the boning knife family that is used to filet and prepare fish. Fillet knife blades are typically 15 to 28 cm (6 to 11 in) long. This allows them to move easily along the backbone and under the skin of meat.[1]"
answer = "a blunt metal concave knife or a flexible wire tool used to flesh a skin or hide, it is usually stored in a kitchen drawer."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What is a fillet knife?
Context: A fillet knife (also called a filleting knife) is a kitchen knife used for filleting. It gives good control and aids in filleting. It is a very flexible member of the boning knife family that is used to filet and prepare fish. Fillet knife blades are typically 15 to 28 cm (6 to 11 in) long. This allows them to move easily along the backbone and under the skin of meat.[1]
Answer: a blunt metal concave knife or a flexible wire tool used to flesh a skin or hide, it is usually stored in a kitchen drawer.
Answer generated by BERT
Answer: a kitchen knife used for filleting



In [76]:
# question 29 - fish slice
# source: https://en.wikipedia.org/wiki/Fish_slice

question = "What is a fish slice?"
context = "A fish slice or metal spatula, in British English, is a kitchen tool with a wide, flat blade with long holes in it, used for lifting and turning food while cooking.[1] In the US, it is regarded as a type of spatula and may be called a turner[2] or flipper.[3]"
answer = "a metal or plastic kitchen tool that has a handle which is bent upward, and a wide, thin blade used for lifting and turning foods on a hot surface, it is usually stored in a kitchen drawer."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What is a fish slice?
Context: A fish slice or metal spatula, in British English, is a kitchen tool with a wide, flat blade with long holes in it, used for lifting and turning food while cooking.[1] In the US, it is regarded as a type of spatula and may be called a turner[2] or flipper.[3]
Answer: a metal or plastic kitchen tool that has a handle which is bent upward, and a wide, thin blade used for lifting and turning foods on a hot surface, it is usually stored in a kitchen drawer.
Answer generated by BERT
Answer: a kitchen tool with a wide , flat blade with long holes in it



In [77]:
# question 30 - bottle opener
# source: https://en.wikipedia.org/wiki/Bottle_opener

question = "What is a bottle opener?"
context = "A bottle opener is a device that enables the removal of metal bottle caps from glass bottles. More generally, it might be thought to include corkscrews used to remove cork or plastic stoppers from wine bottles. A metal bottle cap is affixed to the rim of the neck of a bottle by being pleated or ruffled around the rim. A bottle opener is a specialized lever inserted beneath the pleated metalwork, which uses a point on the bottle cap as a fulcrum on which to pivot."
answer = "a tool used to remove metal tops from some bottles, it is usually stored in a kitchen draw."

print("Question: "+question)
print("Context: "+context)
print("Answer: "+answer)
print("Answer generated by BERT")
answerQuestion(question, context)
print("")

Question: What is a bottle opener?
Context: A bottle opener is a device that enables the removal of metal bottle caps from glass bottles. More generally, it might be thought to include corkscrews used to remove cork or plastic stoppers from wine bottles. A metal bottle cap is affixed to the rim of the neck of a bottle by being pleated or ruffled around the rim. A bottle opener is a specialized lever inserted beneath the pleated metalwork, which uses a point on the bottle cap as a fulcrum on which to pivot.
Answer: a tool used to remove metal tops from some bottles, it is usually stored in a kitchen draw.
Answer generated by BERT
Answer: a device that enables the removal of metal bottle caps from glass bottles

